# CSCI 4253 / 5253 - Lab #4 - Patent Problem with Spark RDD
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

This [Spark cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf) is useful

In [108]:
from pyspark import SparkContext, SparkConf
import numpy as np
import operator

In [109]:
conf=SparkConf().setAppName("Lab4-rddd").setMaster("local[*]")
sc = SparkContext(conf=conf)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=Lab4-rddd, master=local[*]) created by __init__ at <ipython-input-2-0672f4b0797f>:2 

Using PySpark and RDD's on the https://coding.csel.io machines is very slow -- most of the code is executed in Python and this is much less efficient than the java-based code using the PySpark dataframes. Be patient and trying using `.cache()` to cache the output of joins. You may want to start with a reduced set of data before running the full task.

To that end, we've included code to just extract the last 200,000 lines of each file below using the Python "slice" notation. Using that subset of the data your "new patent" table should look like:

![Top partial 10 RDD self-state citations](top-subsample-rdd.png)

When you're ready to run the whole thing, just include all the data and run it again (...and wait...).

These two RDD's are called "rawCitations" and "rawPatents" because you probably want to process them futher (e.g. convert them to integer types, etc). If you haven't used Python "byte" types before, google it. You can convert a byte variable `x` into e.g. a UTF8 string using `x.decode('uft-8')`.

In [277]:
import gzip
with gzip.open('cite75_99.txt.gz', 'r') as f:
    rddCitations = sc.parallelize( f.readlines()[-800000:] )

In [278]:
with gzip.open('apat63_99.txt.gz', 'r') as f:
    rddPatents = sc.parallelize( f.readlines()[-800000:] )

In [280]:
def formatOutput(x):
    x[1][1].append(x[1][0])   
    return(x[0],x[1][1])

rddPatents = rddPatents.map(lambda x: x.decode('utf-8')).map(lambda x: (int(x.split(",")[0]),x.strip().split(",")[1:]))
rddCitations = rddCitations.map(lambda x: x.decode('utf-8')).map(lambda x: (int(x.strip().split(",")[1]),int(x.split(",")[0])))
result = rddCitations.join(rddPatents)\      
            .map(lambda x:  (x[1][0], (x[0] , x[1][1][4])) if(x[1][1][3] == '"US"') else None)\
            .filter(lambda x: x is not None)\
            .join(rddPatents).map(lambda x:  (x[0],1) if(x[1][1][3] == '"US"' and x[1][1][4] == x[1][0][1]) else None)\
            .filter(lambda x: x is not None)\
            .reduceByKey(lambda x,y : x+y)\
            .join(rddPatents)\
            .map(formatOutput)
result = result.sortBy(lambda x : -x[1][22])
for line in result.take(10):
    print(line)